In [1]:
import pandas as pd
import numpy as np
import altair as alt
import os, random
import gensim
from Bio import SeqIO
from gensim.models import Word2Vec
from sklearn.cluster import KMeans, OPTICS
from sklearn.decomposition import PCA
from sklearn import preprocessing, model_selection, metrics


## Defining paths for each and every omic

In [2]:
path_root_data = os.path.join ('..', 'Data', 'Extracted', 'Databases')
path_all_fasta = os.path.join (path_root_data, 'fasta_files', 'AllBins')
path_normalised_metabolomics = os.path.join (path_root_data, 'Metabolomics', 'Normalised_Tables')
path_model_save_root = 'Saved_models'


In [3]:
num_of_mags = len([i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')])
SEED = 42
END = 25
num_of_workers = 3
random.seed(SEED)
np.random.seed(SEED)

---
# GENOMIC ANALYSIS
---
## MAG-related functions

In [31]:
# Function that splits each genome into k-mers thus creating even longer sentence (MAG)
# It returns tokenized genome i.e. [kmer, kmer,...]
def split_genome (genome, k = 5):
    new_genome = []
    n = len(genome)
    
    if n-k <=0:
        return genome
    else:
        for i in range(n-k):
            new_genome.append(genome[i:i+k])
        
        return new_genome


def vectorize_one_mag (one_mag, w2v_model):
    
    # We have to generate vectors for each word in one MAG and then create vector representation of that MAG
    # by averaging vectors of its words
    zero_vector = np.zeros(w2v_model.vector_size)
    word_vectors = []
    one_mag_vector = []
    
    for sentence in one_mag:
        for word in sentence:
            if word in w2v_model.wv:
                try:
                    word_vectors.append(w2v_model.wv[word])
                except KeyError:
                    print ('Key Error')
                    continue
    
    if word_vectors:
        word_vectors = np.asarray(word_vectors)
        one_mag_vector = word_vectors.mean (axis=0)
    
    else:
        one_mag_vector = zero_vector
    
    return one_mag_vector


# Function that vectorizes a MAG (document) with a pretrained word2vec model. It returns vector representation of a given MAG
# Vectorization is done by averaging word (k-mer) vectors for the whole document (MAG)
def vectorize_mags (w2v_model, end = 25):
    
    print ('Vectorizing MAGs')
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    list_of_mag_vectors = []
    
    # This was done so that I could work with first 'end' FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # Vectorizing MAGs one by one
                list_of_mag_vectors.append (vectorize_one_mag (one_mag, w2v_model))
    
    print ('Finished vectorizing')
    
    return list_of_mag_vectors
    
    

# If one wants to import MAGs in order to vectorize them, one should use start argument in order to skip first 'start' MAGs
# If one wants to import MAGs to train word2vec model, one should use only end argument, so that first 'end' MAGs are used for training
# Todo: Implement randomisation in picking MAGs for training, and don't use first 'start' MAGs for training
# (create list of indexes from 0 to 1364, use sklearn split train test, then traverse directory and use only MAGs with indexes in train for training w2v)
def import_mags_and_build_model (end = 25, path_all_fasta = path_all_fasta):
    
    print ('Importing MAGs')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                # If we do not have a model, we build one
                if i == 0:
                    print ('Building w2v model')
                    # We build our model on the first MAG
                    w2v_model = Word2Vec (sentences = one_mag, size = 100, workers = num_of_workers, seed=SEED)
                
                # Else we just expand its vocabulary
                else:
                    # Now we expand our vocabulary
                    w2v_model.build_vocab (one_mag, update = True)
                    
    print ('Finished building')
    
    return w2v_model


def train_model (w2v_model, epochs, end = 25):
    
    print ('Starting model training')
    
    # There are 1364 MAGs enclosed in FASTA files
    # I have to traverse every FASTA file, and in each file every sequence
    
    fasta_files = [i for i in os.listdir(path_all_fasta) if (i[-2:] == 'fa')]
    
    # This was done so that I could work with first 100 FASTA files only. Otherwise, I should just remove: i, and enumerate
    for i, fasta_file_name in enumerate(fasta_files):
        
        if i == end:
            break
        
        else:
            with open(os.path.join(path_all_fasta, fasta_file_name), 'r') as input_file:
                
                one_mag = []
                for fasta_string in SeqIO.parse(input_file, "fasta"):
                    
                    # Get kmers of a genome and create a sentence (list of words)
                    temp_kmers = split_genome (str(fasta_string.seq))
                    
                    # Create a document (list of sentences)
                    one_mag.append(temp_kmers)
                    
                
                w2v_model.train (one_mag, total_examples = w2v_model.corpus_count, epochs = epochs)
                    
    print ('Model training finished')
    
    return w2v_model


def visualize_with_pca (data, labels, centers):
    
    pca_model = PCA (n_components = 2, random_state = SEED)
    data_transformed = pca_model.fit_transform (data)
    
    data_transformed = pd.DataFrame (data_transformed)
    data_transformed.columns = ['PC_1', 'PC_2']
    data_transformed['Labels'] = labels
    
    chart_data = alt.Chart(data_transformed).mark_circle(opacity = 1).encode(
        alt.X ('PC_1:Q'),
        alt.Y ('PC_2:Q'),
        alt.Color ('Labels:N', legend = alt.Legend())
    )
    
    # This means we are visualising centroids from k_means (there are less centroids that data points)
    if labels.shape[0] != centers.shape[0]:
        
        centers_transformed = pca_model.fit_transform (centers)
        centers_transformed = pd.DataFrame (centers_transformed)
        centers_transformed.columns = ['PC_1', 'PC_2']
        
        chart_centers = alt.Chart(centers_transformed).mark_point(shape = 'diamond', color = 'black', size = 50, opacity = 0.7).encode(
            alt.X ('PC_1:Q'),
            alt.Y ('PC_2:Q'),
        )
        
        return chart_data + chart_centers
    
    # For DBSCAN there are no centroids
    else:
        return chart_data



## MAG examination

In [7]:

# FOR CLUSTERING I SHOULD CREATE A DATAFRAME WITH MAGs INDEXES AND THEIR VECTOR REPRESENTATIONS

final_model = import_mags_and_build_model (end = END, path_all_fasta = path_all_fasta)


Importing MAGs
Building w2v model
Finished building


In [8]:
final_model.wv.most_similar('ACGGC')

[('CCGGC', 0.5897153615951538),
 ('TCGGC', 0.560882568359375),
 ('ACGGA', 0.5486097931861877),
 ('GCGGC', 0.5190006494522095),
 ('ACGGG', 0.5120842456817627),
 ('ACGGT', 0.48747333884239197),
 ('ACGAA', 0.31071770191192627),
 ('ACGAC', 0.3037322759628296),
 ('ACGTC', 0.29194021224975586),
 ('ACGTT', 0.28757983446121216)]

In [9]:
# Train model. It tooks ~10 minutes for END = 25 amount of MAGs
EPOCHS = 10

final_model = train_model (final_model, epochs = EPOCHS, end = END)


Starting model training
Model training finished


In [10]:
final_model.wv.save_word2vec_format(os.path.join (path_model_save_root, 'model_25.bin'), binary=True) 

Now I should vectorize documents with this model. For further use, I could save this model's weights, and use it to vectorize all mags. That would take a lot, but every MAG will have its vector representation
> This could be done by importing one MAG at a time, then tokenizing it (like before), then getting vector representations of that MAG's sentences (genomes) and then finding the vector representation of the whole MAG (document). If I do that for one MAG at a time, There is no need to worry about memory


In [11]:
list_of_mag_vectors = vectorize_mags (final_model, end = END)

Vectorizing MAGs
Finished vectorizing


In [12]:
list_of_mag_vectors[0:2]

[array([-0.3412855 , -1.3074803 ,  1.6673468 , -0.18193825,  0.6783584 ,
         0.5612512 ,  0.24554321,  1.012197  ,  0.79908276, -0.33281672,
        -0.15648904,  1.0270334 , -1.7591202 ,  1.6900983 , -1.4474658 ,
         0.14639385,  0.2635378 , -1.107245  , -0.60734785, -0.828628  ,
        -1.6702052 , -2.5707538 , -0.855652  , -0.9473689 ,  1.5789722 ,
        -0.12598857, -0.56086534,  0.6502867 , -0.45995945, -0.21470664,
        -0.59516215,  0.10712713, -0.92095405, -0.44383943,  0.4565169 ,
        -1.2524884 ,  0.0095067 ,  0.3513306 ,  1.5343064 , -0.72287744,
        -0.5809037 ,  1.7036309 ,  2.6705778 , -0.5011542 ,  0.997374  ,
         0.0991397 , -1.9106972 , -0.14362767, -0.38712674,  1.3254927 ,
         1.7431389 ,  2.058191  , -0.16283734, -0.07293132, -0.7716484 ,
        -1.209459  , -1.3840557 , -1.1185156 ,  0.85279655, -0.21090718,
        -0.94948244,  1.3082253 , -0.264265  , -0.6213532 ,  1.2397407 ,
         0.424508  ,  0.5835601 ,  0.49468315,  1.5

In [13]:
mags_df = pd.DataFrame (list_of_mag_vectors)
mags_df


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.341285,-1.307480,1.667347,-0.181938,0.678358,0.561251,0.245543,1.012197,0.799083,-0.332817,...,0.411772,0.342497,0.645314,-0.009584,1.261419,0.947777,1.005131,-1.612071,0.664969,0.637182
1,-0.353241,1.356478,-1.519977,0.489819,0.011351,-1.753141,0.638735,0.260151,-0.848185,0.125419,...,0.086121,0.107420,1.613271,1.648110,0.013657,-1.203047,-0.610382,0.529846,-0.837352,-2.192798
2,-1.027619,-1.867537,1.986852,-0.700268,0.068323,1.246489,0.360177,1.379367,0.769427,-0.346021,...,-0.694683,0.318921,0.516075,-0.900386,1.879338,1.581902,0.321573,-1.432031,0.769141,0.652904
3,0.075040,1.481463,-2.178516,0.583901,-0.200483,-1.770640,0.561849,-0.078156,-0.928566,0.127184,...,0.284476,0.141210,1.831504,1.398362,-0.607934,-0.966220,-0.872207,0.923392,-0.848844,-2.399399
4,0.075354,1.427001,-2.124519,0.521798,-0.294279,-1.672292,0.562149,0.005005,-0.853643,0.065581,...,0.274580,0.118297,1.762440,1.379492,-0.546828,-0.996493,-0.796771,0.906181,-0.835940,-2.342209
5,0.129941,1.277619,-1.844996,0.219384,-0.353005,-1.293824,0.321701,0.232485,-0.444988,-0.437443,...,0.159732,-0.018819,1.726185,0.603282,0.019392,-0.371690,-0.961833,0.956784,-0.973640,-2.271333
6,-0.989811,-2.985559,2.709233,-0.640486,0.232940,1.623222,0.477722,1.724624,0.602666,-0.209730,...,-1.317823,0.606889,0.257215,-0.741542,2.227737,1.319617,1.197607,-2.712788,1.819405,2.038449
7,-0.951616,-2.665345,2.896326,-0.725836,0.373385,1.783689,0.521206,1.706788,0.528568,-0.127032,...,-0.642154,0.893542,0.116415,-0.858817,1.663370,1.752637,1.220060,-2.411797,1.615608,1.523981
8,0.063258,-0.265686,0.631984,-0.196667,0.216028,0.246256,0.385285,0.877592,0.378182,-0.462380,...,0.465767,0.268095,0.891215,-0.241772,1.045041,0.846471,0.344685,-0.573762,-0.044410,-0.331948
9,-0.025258,-0.052589,-0.332469,-0.216357,-0.029103,-0.398063,0.019306,0.645865,0.542800,-0.684887,...,0.247554,0.156420,1.397167,-0.127547,1.007807,0.455250,-0.290012,-0.409483,-0.254392,-0.798456


## Data preprocessing

In [14]:
mag_scaler = preprocessing.StandardScaler()
scaled_mags_df = mag_scaler.fit_transform(mags_df)

scaled_mags_df

array([[-0.43592917, -0.94533422,  1.1326052 , ..., -1.11089311,
         0.86863874,  0.97492981],
       [-0.46306863,  0.77474429, -0.64966773, ...,  0.54099618,
        -0.65185799, -0.98656637],
       [-1.99393027, -1.30695476,  1.31126462, ..., -0.97204231,
         0.9740716 ,  0.98582712],
       ...,
       [-1.01060177, -1.58150372,  1.44552636, ..., -1.74310989,
         1.62544066,  1.88656041],
       [ 1.22936975,  0.96140213, -0.40522044, ...,  1.14386822,
        -1.359008  , -0.42331992],
       [ 2.12990612,  0.9193756 , -1.4250071 , ...,  1.11983122,
        -0.5186798 , -1.18943218]])

## Clustering

### 1. K-means

In [15]:
k_range_end = 15 # Usually it is sqrt(# of mags)

k_range = range(1, k_range_end)

k_mean_models = [KMeans (n_clusters = i, random_state = SEED) for i in k_range]
k_scores = [k_mean_model.fit(scaled_mags_df).score(scaled_mags_df) for k_mean_model in k_mean_models]
k_data = pd.DataFrame ({'k_range':k_range, 'k_scores':k_scores})
k_data


,k_range,k_scores
0,1,-2500.000000
1,2,-987.169696
2,3,-635.763126
3,4,-519.369088
4,5,-412.674751
5,6,-337.076059
6,7,-281.966621
7,8,-243.332138
8,9,-208.154294
9,10,-173.791454


In [16]:
k_num_chart = alt.Chart(data = k_data).mark_line().encode(
    alt.X ('k_range:Q'),
    alt.Y ('k_scores:Q')
)

k_num_chart

alt.Chart(...)

In [17]:
# We can see from the chart above that 6 or 7 clusters are optimal for this task (where END = 25 MAGs)
num_of_clusters = 7

k_means_model = KMeans (n_clusters = num_of_clusters, random_state = SEED)
k_means_predicted = k_means_model.fit_predict(scaled_mags_df)
k_means_predicted

array([1, 0, 3, 0, 0, 2, 3, 3, 4, 4, 4, 1, 4, 2, 2, 5, 0, 5, 2, 1, 4, 0,
       3, 6, 2], dtype=int32)

In [32]:
k_means_chart = visualize_with_pca (scaled_mags_df, k_means_predicted, k_means_model.cluster_centers_)
k_means_chart


alt.LayerChart(...)

### 2. OPTICS

In [19]:
MIN_SAMPLES = 3

optics_model = OPTICS (min_samples = MIN_SAMPLES, n_jobs = num_of_workers)
optics_predicted = optics_model.fit_predict (scaled_mags_df)
optics_predicted

array([ 0,  1,  2,  1,  1,  1,  2,  2,  0,  0,  0,  0,  0,  1,  1,  1,  1,
        1,  1,  0,  0,  1,  2, -1,  1])

In [33]:
# Visualize clusters, since there are no centroids, we are sending bogus array
optics_chart = visualize_with_pca (scaled_mags_df, optics_predicted, np.empty([optics_predicted.shape[0], 1], dtype=int))
optics_chart


alt.Chart(...)

In [35]:
# Side by side comparison
alt.hconcat (k_means_chart, optics_chart).resolve_scale(color='independent')

alt.HConcatChart(...)

## Evaluation

In [39]:
eval_k_means = metrics.silhouette_score (scaled_mags_df, k_means_predicted)
eval_optics = metrics.silhouette_score (scaled_mags_df, optics_predicted)

print ('Silhouette scores: [best = 1, worst = -1]')
print ('\t1. K-means:', eval_k_means)
print ('\t2. OPTICS:', eval_optics)

Silhouette scores: [best = 1, worst = -1]
	1. K-means: 0.3284530838720724
	2. OPTICS: 0.4068072149015211


---
# METABOLOMIC ANALYSIS
---
## Importing Metabolomic data

In [60]:
metabolomics_file_name = os.path.join(path_normalised_metabolomics, os.listdir(path_normalised_metabolomics)[0])
metabolomics_df = pd.read_csv (metabolomics_file_name, delimiter = '\t')
metabolomics_df

,Metabolite,tp,date,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,!Phosphoric_acid_3TMS,D1,2012-04-17,snp,known,0.757113,0.737943,0.098650,3,0.056956,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
1,!Phosphoric_acid_3TMS,D10,2012-01-25,snp,known,0.720935,0.873509,0.266593,3,0.153918,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
2,!Phosphoric_acid_3TMS,D11,2012-03-22,snp,known,0.625047,0.652268,0.057820,3,0.033382,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
3,!Phosphoric_acid_3TMS,D12,2012-01-19,snp,known,0.964956,0.941878,0.053754,3,0.031035,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
4,!Phosphoric_acid_3TMS,D13,2011-05-13,snp,known,0.963984,0.974479,0.071659,3,0.041373,4.302653,extracellular,nonpolar,C00009,phosphoric acid,phosphoric acid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,Valine_2TMS,D51,2011-08-11,bp,known,0.704821,0.478251,0.428368,3,0.247319,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18866,Valine_2TMS,D6,2011-06-03,bp,known,1.085888,1.197471,0.487484,3,0.281449,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18867,Valine_2TMS,D7,2012-04-04,bp,known,2.958023,2.813145,0.341213,3,0.197000,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine
18868,Valine_2TMS,D8,2012-02-08,bp,known,2.515980,2.322248,0.618556,3,0.357123,4.302653,intracellular,polar,"C16436,C06417,C00183",valine;D-valine;L-valine,valine;D-valine;L-valine


## Data preprocessing

In [61]:
metabolomics_df['date'] = pd.to_datetime(metabolomics_df['date'])
metabolomics_df.insert (0, 'date', metabolomics_df.pop('date'))
metabolomics_df.sort_values ('date', inplace = True, ignore_index = True)
metabolomics_df

,date,Metabolite,tp,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,2011-03-21,Leucine_2TMS,D37,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar,"C16439,C00123,C01570",leucine;L-leucine;D-leucine,leucine;L-leucine;D-leucine
1,2011-03-21,No match: 1181.78_EM_SP_D1_1_223,D37,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar,NaN,NaN,NaN
2,2011-03-21,unknown#emu_WW_2144.25,D37,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar,NaN,NaN,NaN
3,2011-03-21,Unknown#bth_pae_001,D37,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar,NaN,NaN,NaN
4,2011-03-21,No match: 2220.34_EM_BP_D1_1_192,D37,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,Octadecanol_1TMS,D48,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar,D01924,octadecan-1-ol,octadecan-1-ol
18866,2012-05-03,No match: 1447.93_EM_BP_D2_1_57,D48,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar,NaN,NaN,NaN
18867,2012-05-03,Lactulose_8TMS,D48,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar,C07064,lactulose,lactulose
18868,2012-05-03,No match: 2004.78_EM_BNP_D10_1_205,D48,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar,NaN,NaN,NaN


In [62]:
print ('Dataset uniqueness:')
print ('\t1. Metabolites:', len(np.unique(metabolomics_df['Metabolite'].values)))
print ('\t2. Types:', len(np.unique(metabolomics_df['type'].values)))
print ('\t3. Known types:', len(np.unique(metabolomics_df['known_type'].values)))
print ('\t2. Ns:', len(np.unique(metabolomics_df['N'].values)))
print ('\t2. Type 2s', len(np.unique(metabolomics_df['type2'].values)))
print ('\t2. Measurements', len(np.unique(metabolomics_df['measurement'].values)))

Dataset uniqueness:
	1. Metabolites: 313
	2. Types: 4
	3. Known types: 2
	2. Ns: 2
	2. Type 2s 2
	2. Measurements 2


In [63]:
# Changing metabolite name if it is unknown
metabolomics_df.loc[metabolomics_df['known_type'].eq('unknown'), 'Metabolite'] = 'Unknown'
metabolomics_df

,date,Metabolite,tp,type,known_type,means,medians,sds,N,se,ci,type2,measurement,KEGG.Compound.ID,Chebi.Name,Chebi.Name_combined
0,2011-03-21,Leucine_2TMS,D37,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar,"C16439,C00123,C01570",leucine;L-leucine;D-leucine,leucine;L-leucine;D-leucine
1,2011-03-21,Unknown,D37,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar,NaN,NaN,NaN
2,2011-03-21,Unknown,D37,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar,NaN,NaN,NaN
3,2011-03-21,Unknown,D37,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar,NaN,NaN,NaN
4,2011-03-21,Unknown,D37,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,Octadecanol_1TMS,D48,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar,D01924,octadecan-1-ol,octadecan-1-ol
18866,2012-05-03,Unknown,D48,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar,NaN,NaN,NaN
18867,2012-05-03,Lactulose_8TMS,D48,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar,C07064,lactulose,lactulose
18868,2012-05-03,Unknown,D48,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar,NaN,NaN,NaN


In [64]:
# Saving the name column and removing unnecessairy columns
metabolite_names = metabolomics_df['Metabolite']
metabolomics_df.drop(labels = ['Metabolite', 'tp', 'KEGG.Compound.ID', 'Chebi.Name', 'Chebi.Name_combined'], axis = 1, inplace = True)
metabolomics_df

,date,type,known_type,means,medians,sds,N,se,ci,type2,measurement
0,2011-03-21,sp,known,1.440122,1.489979,0.146440,3,0.084547,4.302653,extracellular,polar
1,2011-03-21,sp,unknown,3.778469,3.801417,1.924377,3,1.111040,4.302653,extracellular,polar
2,2011-03-21,bnp,unknown,0.949545,0.947209,0.065422,3,0.037771,4.302653,intracellular,nonpolar
3,2011-03-21,bp,unknown,2.095116,1.809891,1.916896,3,1.106721,4.302653,intracellular,polar
4,2011-03-21,bp,unknown,1.631643,1.571125,0.168501,3,0.097284,4.302653,intracellular,polar
...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,bnp,known,1.364105,1.457780,0.170311,3,0.098329,4.302653,intracellular,nonpolar
18866,2012-05-03,bp,unknown,4.815774,1.160165,6.835474,3,3.946463,4.302653,intracellular,polar
18867,2012-05-03,bp,known,0.697489,0.684339,0.049632,3,0.028655,4.302653,intracellular,polar
18868,2012-05-03,bnp,unknown,0.785996,0.744529,0.139901,3,0.080772,4.302653,intracellular,nonpolar


In [65]:
# Dummy eencoding categorical data
metabolomics_df = pd.get_dummies(metabolomics_df, columns = ['type', 'known_type', 'N', 'type2', 'measurement'])
metabolomics_df

,date,means,medians,sds,se,ci,type_bnp,type_bp,type_snp,type_sp,known_type_known,known_type_unknown,N_2,N_3,type2_extracellular,type2_intracellular,measurement_nonpolar,measurement_polar
0,2011-03-21,1.440122,1.489979,0.146440,0.084547,4.302653,0,0,0,1,1,0,0,1,1,0,0,1
1,2011-03-21,3.778469,3.801417,1.924377,1.111040,4.302653,0,0,0,1,0,1,0,1,1,0,0,1
2,2011-03-21,0.949545,0.947209,0.065422,0.037771,4.302653,1,0,0,0,0,1,0,1,0,1,1,0
3,2011-03-21,2.095116,1.809891,1.916896,1.106721,4.302653,0,1,0,0,0,1,0,1,0,1,0,1
4,2011-03-21,1.631643,1.571125,0.168501,0.097284,4.302653,0,1,0,0,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,1.364105,1.457780,0.170311,0.098329,4.302653,1,0,0,0,1,0,0,1,0,1,1,0
18866,2012-05-03,4.815774,1.160165,6.835474,3.946463,4.302653,0,1,0,0,0,1,0,1,0,1,0,1
18867,2012-05-03,0.697489,0.684339,0.049632,0.028655,4.302653,0,1,0,0,1,0,0,1,0,1,0,1
18868,2012-05-03,0.785996,0.744529,0.139901,0.080772,4.302653,1,0,0,0,0,1,0,1,0,1,1,0


In [66]:
# Standardizing data
scaled_metabolomics_df = metabolomics_df
metabolomics_scaler = preprocessing.StandardScaler()
scaled_metabolomics_df[['means', 'medians', 'sds', 'se', 'ci']] = metabolomics_scaler.fit_transform(metabolomics_df[['means', 'medians', 'sds', 'se', 'ci']])

scaled_metabolomics_df

,date,means,medians,sds,se,ci,type_bnp,type_bp,type_snp,type_sp,known_type_known,known_type_unknown,N_2,N_3,type2_extracellular,type2_intracellular,measurement_nonpolar,measurement_polar
0,2011-03-21,-0.145146,-0.094812,-0.189837,-0.190047,-0.128175,0,0,0,1,1,0,0,1,1,0,0,1
1,2011-03-21,0.148026,0.316083,-0.000023,-0.000270,-0.128175,0,0,0,1,0,1,0,1,1,0,0,1
2,2011-03-21,-0.206653,-0.191297,-0.198486,-0.198695,-0.128175,1,0,0,0,0,1,0,1,0,1,1,0
3,2011-03-21,-0.063026,-0.037942,-0.000822,-0.001068,-0.128175,0,1,0,0,0,1,0,1,0,1,0,1
4,2011-03-21,-0.121134,-0.080386,-0.187482,-0.187692,-0.128175,0,1,0,0,0,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18865,2012-05-03,-0.154677,-0.100535,-0.187288,-0.187499,-0.128175,1,0,0,0,1,0,0,1,0,1,1,0
18866,2012-05-03,0.278079,-0.153441,0.524288,0.523941,-0.128175,0,1,0,0,0,1,0,1,0,1,0,1
18867,2012-05-03,-0.238254,-0.238027,-0.200172,-0.200380,-0.128175,0,1,0,0,1,0,0,1,0,1,0,1
18868,2012-05-03,-0.227158,-0.227327,-0.190535,-0.190745,-0.128175,1,0,0,0,0,1,0,1,0,1,1,0


## Clustering

In [ ]:
# Deep learning temporal clustering